In [1]:
from tkinter import *
from PIL import Image, ImageTk
import os
from math import floor
path = os.path.join(os.curdir, 'images')
path

'./images'

In [3]:
ims_path = os.listdir(path)
samp = os.path.join(path, 'samples')
os.mkdir(samp)
nothing = os.path.join(path, 'nothing')
os.mkdir(nothing)
print(samp, nothing)

./images/samples ./images/nothing


In [5]:
class Searching_Lines(Tk):
    dots = []
    img_number = 0
    x = 0
    y = 0
    R = 0
    
    def __init__(self, filename):
        Tk.__init__(self)
        self.width = int(self.winfo_screenwidth() * 0.8)
        self.height = int(self.winfo_screenheight() * 0.8)
        
        self.ratio = float(self.width) / self.height
        self.title('Find lines')
        
        self.photos = [os.path.join(path, i) for i in ims_path]
        self.result_file = open(filename, 'w')
        
        self.canvas = Canvas(self, width=self.width, height=self.height, bg="white")
        self.canvas.pack(side="top", fill="both", expand=True)
        self.canvas.bind("<ButtonPress-1>", self.on_button_press)
        self.bind("1", self.check)
        self.bind("<Escape>", self.back)
        self.bind("<Tab>", self.next)
        
        self.draw_image()

    def draw_image(self):
        self.dots.clear()
        self.image = Image.open(self.photos[self.img_number])
        w,h = self.image.size
        ratio = float(w)/h
        
        if ratio > self.ratio:
            self.coef = float(w) / self.width
            size = (self.width, int(floor(h / self.coef)))

        if ratio < self.ratio:
            self.coef = float(h) / self.height
            size = (int(floor(w / self.coef)), self.height)

        if ratio == self.ratio:
            self.coef = float(im_width / self.width)
            size = (self.width, self.height)
        
        self.resized = self.image.resize(size, Image.ANTIALIAS)
        self.tk_image = ImageTk.PhotoImage(self.resized)
        self.canvas.create_image(0, 0, anchor="nw", image=self.tk_image)
        
        
    def check(self, event):
        x12 = self.dots[0][0] - self.dots[1][0]
        x23 = self.dots[1][0] - self.dots[2][0]
        x31 = self.dots[2][0] - self.dots[0][0]
        y12 = self.dots[0][1] - self.dots[1][1]
        y23 = self.dots[1][1] - self.dots[2][1]
        y31 = self.dots[2][1] - self.dots[0][1]
        z1 = self.dots[0][0] ** 2 + self.dots[0][1] ** 2
        z2 = self.dots[1][0] ** 2 + self.dots[1][1] ** 2
        z3 = self.dots[2][0] ** 2 + self.dots[2][1] ** 2
        zx = y12 * z3 + y23 * z1 + y31 * z2
        zy = x12 * z3 + x23 * z1 + x31 * z2
        z = x12 * y31 - y12 * x31
        self.x = (-1) * zx / (2 * z)
        self.y = zy / (2 * z)
        a = (x12 ** 2 + y12 **2) ** 0.5
        b = (x23 ** 2 + y23 **2) ** 0.5
        c = (x31 ** 2 + y31 **2) ** 0.5
        p = (a + b + c) / 2
        S = (p * (p - a) * (p - b) * (p - c)) ** 0.5
        self.R = a * b * c / (4 * S)
        self.canvas.create_oval(self.x - self.R, self.y - self.R,
                               self.x + self.R, self.y + self.R,
                                fill='Red', outline="Red")
    
    def back(self, event):
        self.canvas.delete()
        self.dots.clear()
        self.draw_image()
        
    def on_button_press(self, event):
            self.dots.append([event.x, event.y])
        
            
    def f(self, x):
        return self.coef*x
    
    def next(self, event):
        cur = os.path.join(path, ims_path[self.img_number])
        if len(self.dots) != 0:
            self.result_file.write(ims_path[self.img_number])
            self.result_file.write('\n\n')
            self.result_file.write(str(self.x) + '\n' + str(self.y) + '\n' + str(self.R) + '\n')
            dest = os.path.join(samp, ims_path[self.img_number])
            os.rename(cur, dest)
        else:
            dest = os.join.path(nothing, ims_path[self.img_number])
            os.rename(cur, dest)
        self.img_number += 1
        if self.img_number >= len(self.photos):
            self.result_file.close()
            self.destroy()
        else:
            self.draw_image()

        
if __name__ == '__main__':
    app = Searching_Lines(os.path.join(samp, 'dataset.txt'))
    app.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/usr/lib/python3.6/tkinter/__init__.py", line 1702, in __call__
    return self.func(*args)
  File "<ipython-input-5-c992a1928747>", line 89, in next
    cur = os.join.path(path, ims_path[self.img_number])
AttributeError: module 'os' has no attribute 'join'
Exception in Tkinter callback
Traceback (most recent call last):
  File "/usr/lib/python3.6/tkinter/__init__.py", line 1702, in __call__
    return self.func(*args)
  File "<ipython-input-5-c992a1928747>", line 89, in next
    cur = os.join.path(path, ims_path[self.img_number])
AttributeError: module 'os' has no attribute 'join'
Exception in Tkinter callback
Traceback (most recent call last):
  File "/usr/lib/python3.6/tkinter/__init__.py", line 1702, in __call__
    return self.func(*args)
  File "<ipython-input-5-c992a1928747>", line 89, in next
    cur = os.join.path(path, ims_path[self.img_number])
AttributeError: module 'os' has no attribute 'join'
Excepti